In [1]:
from langchain.llms import OpenAI

In [2]:
import os
os.environ["OPEN_API_KEY"] = "ADD YOUR KEY"

#### Create Your LLM Model

In [3]:
# llm = OpenAI(temperature=0.6)
# This throws error for key not found

In [4]:
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature=0.6)

- Temperature => Decides creativity of our model. 0 (low risk) to close to 1 (high risk) of prediction

In [5]:
text = "What is the Capital of India"
print(llm.predict(text))



The capital of India is New Delhi.


2. How to deal with Hugging face open source models

In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "ADD YOUR KEY"

In [7]:
from langchain import HuggingFaceHub

Langchain is just used as a wrapper, it can call OpenAI, LLM models, Hugging face models.

In [8]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={})

d:\MyStudy\llm\llm_langchain_openai_tutorials\krish_langchain_series\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\MyStudy\llm\llm_langchain_openai_tutorials\krish_langchain_series\Langchain\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [9]:
output = llm_huggingface.predict("Can you tell me the capital of India?")
print(output)

calcutta


Here you can see, Hugging face is giving a short answer and the answer may not be correct always.

In [10]:
output = llm_huggingface.predict("Can you write poem about rain?")
print(output)

rain is a beautiful thing


In [11]:
llm.predict("Can you write poem about rain?")

"\n\nRain falls from the sky\nIt fills the air with its sighs\nThe water droplets make their way\nDown to the earth, and here they stay\n\nThe sound of its patter\nEchoes throughout the land\nBringing life to the soil\nAnd nourishment to the sand\n\nThe grey clouds that bring it\nAre a reminder of its might\nThat it can come down hard\nOr be a gentle light\n\nRain is a blessing\nIt's beauty, we can't ignore\nIt's power, we can't deny\nAs it washes us to the core"

We can observe the llm model gives better response than hugging face models. (The paid hugging face models can also give better results)

Prompt Templates and LLM Chain:
- To define how our input and output should be.

In [12]:
from langchain.prompts import PromptTemplate

In [13]:
prompt_template = PromptTemplate (input_variables=['country'],
                                  template = "Tell me the capital of the country {country}")

prompt_template.format(country = "India")    

'Tell me the capital of the country India'

In [14]:
# Following is not the correct way to give input in a template, it throws error
# llm.predict(prompt_template=prompt_template, text=["India "])

In [15]:
from langchain.chains import LLMChain

In [16]:
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.run("India"))



The capital of India is New Delhi.


Combining Multiple Chains using Simple Sequential Chain

In [17]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm, prompt = capital_prompt)

# creating one more template with same input
famous_template = PromptTemplate(input_variables=['capital'],
                                 template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [18]:
# To give the template one after another to next prompt, will use Sequential chain
from langchain.chains import SimpleSequentialChain

In [19]:
chain = SimpleSequentialChain(chains= [capital_chain, famous_chain ])
print(chain.run("India"))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-P3zz0utKN7KVRgIRbLktoL1Z on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-P3zz0utKN7KVRgIRbLktoL1Z on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l

 Here are some amazing places to visit in New Delhi: 

1. Red Fort: The majestic Red Fort is a UNESCO World Heritage Site and one of the iconic landmarks of India. It is a symbol of Mughal grandeur and a reminder of India’s colonial past. 

2. Qutub Minar: The world-famous Qutub Minar is a 73-meter tall tower constructed by Qutub-ud-din Aibak in 1193. This is a UNESCO World Heritage Site and one of the most visited tourist sites in India. 

3. Humayun's Tomb: This is a beautiful mausoleum built by Humayun's wife in 1570. It is a UNESCO World Heritage Site and an important landmark in Indian history. 

4. Jama Masjid: This is one of the largest mosques in India and is a popular tourist site. It was built by Mughal Emperor Shah Jahan in 1656. 

5. India Gate: India Gate is a war memorial located in the heart of New Delhi. It is a popular tourist spot and a symbol of India's freedom struggle. 

6. Akshardham Temple: This is


The above output didnt tell us the specific sentence answer for our first template. For that we have to use buffer memory

Sequential Chain

In [20]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm, prompt = capital_prompt, output_key="capital")

famous_template = PromptTemplate(input_variables=['capital'],
                                 template = "Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [21]:
from langchain.chains import SequentialChain

In [22]:
chain = SequentialChain(chains= [capital_chain, famous_chain ],
                        input_variables=['country'],
                        output_variables=['capital', 'places'])
# print(chain.run("India"))   #for sequential simple chain.run is not used
print(chain({'country':"India"}))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-P3zz0utKN7KVRgIRbLktoL1Z on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-P3zz0utKN7KVRgIRbLktoL1Z on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.', 'places': " Here are some amazing places to visit in New Delhi:\n\n1. Red Fort: This majestic fort stands as a symbol of India's rich cultural and architectural heritage.\n\n2. India Gate: This iconic monument is a memorial to the Indian soldiers who lost their lives in World War I.\n\n3. Jama Masjid: This is one of the largest mosques in India and is a must-visit for its intricate architecture and vibrant atmosphere.\n\n4. Humayun's Tomb: This magnificent Mughal era structure is a World Heritage Site and a must-visit to appreciate its beauty.\n\n5. Lodhi Gardens: This sprawling garden is a great place to relax and enjoy nature.\n\n6. Qutub Minar: This is the tallest brick minaret in the world and is a great place to explore and admire its architectural beauty.\n\n7. Akshardham Temple: This is a modern Hindu temple complex and is a must-visit for its beautiful architecture and spiritual atmosphere.\n\n8. Chandni 

Chatmodels with ChatOpenAI

In [23]:
from langchain.chat_models import ChatOpenAI

In [24]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [25]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature=0.6, model = 'gpt-3.5-turbo')

In [26]:
# Create schema in a list

chatllm([
SystemMessage(content= "You are an AIML Interviewer"),
HumanMessage(content= "Please provide some questions about AIML")
])

AIMessage(content='1. What is AIML and what does it stand for?\n2. How does AIML differ from other programming languages?\n3. What are the main components of an AIML file?\n4. How does AIML utilize pattern matching and template responses?\n5. Can you explain the concept of categories in AIML?\n6. How does AIML handle user inputs and generate appropriate responses?\n7. Can you describe the AIML pattern matching algorithm?\n8. What are some advantages of using AIML for creating chatbots or virtual assistants?\n9. Are there any limitations or challenges associated with using AIML?\n10. How can AIML be extended or customized to improve its functionality?\n11. Can you provide examples of AIML code snippets and explain their purpose?\n12. How does AIML handle context and maintain conversation flow?\n13. What are some popular AIML platforms or frameworks available in the market?\n14. Can AIML be used for natural language processing and understanding?\n15. How does AIML handle ambiguous or ope

Prompt Template + LLM + Output Parser

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [28]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(" , ")

In [29]:
# define template to make comma separated string
template = " Your are a helpful assisatent. When the user gives any input, you should generate 5 relatable words which should ne comma separated."
human_template = "{text}"
chatpromt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [30]:
chain = chatpromt|chatllm|Commaseparatedoutput()    #chain up operation


In [31]:
chain.invoke({"text":"Intelligent"})

['Smart, Clever, Brilliant, Genius, Wise']